# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.8688874961220803),
 (1, 1.8688874961220803),
 (2, 1.8688874961220803),
 (3, 1.8688874961220803),
 (4, 1.8688874961220803)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18), ('it', 18)]),
 (1, [('a', 2), ('what', 10)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.00198678486097692),
   (None, 0.006832783133461806),
   (None, 0.013265848685509063),
   (None, 0.11404270300807795),
   (None, 0.19404816683447912),
   (None, 0.31024034566130854),
   (None, 0.34951331627080995),
   (None, 0.35031889917256875),
   (None, 0.37280198638233564),
   (None, 0.44126488637263983),
   (None, 0.48499603318347007),
   (None, 0.4993508073326156)]),
 (1,
  [(None, 0.5432580162764701),
   (None, 0.5565859750002844),
   (None, 0.5571697327230147),
   (None, 0.5622040148376836),
   (None, 0.5886245404372275),
   (None, 0.646036984168623),
   (None, 0.6523320960080421),
   (None, 0.6547150466577952),
   (None, 0.6693101362317809),
   (None, 0.7132965826252247),
   (None, 0.791713358307749),
   (None, 0.8101865452837641),
   (None, 0.8234858895654715),
   (None, 0.8269503180750427),
   (None, 0.8384533736811259),
   (None, 0.840050279336903),
   (None, 0.8517931931472374),
   (None, 0.8862746956665395)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
from typing import Iterator
import numpy as np

np.random.seed(42)
numbers = np.random.randint(low=-50, high=200, size=50).tolist()

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for x in split:
            yield (None, x)

    split_size = int(np.ceil(len(numbers) / maps))
    for i in range(0, len(numbers), split_size):
        yield RECORDREADER(numbers[i:i + split_size])

def MAP(_, x: int):
    # Отправляем все значения под один ключ, чтобы найти глобальный максимум
    yield ("max", x)

def REDUCE_MAX(key: str, xs: Iterator[int]):
    m = None
    for x in xs:
        m = x if m is None else max(m, x)
    yield (key, m)

# COMBINER снижает объём данных, отправляемых по сети: на каждый map-сплит уходит 1 значение
partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE_MAX, COMBINER=REDUCE_MAX
)
partitioned_output = [(pid, list(part)) for (pid, part) in partitioned_output]

result = [kv for (pid, part) in partitioned_output for kv in part]
result, "python max =", max(numbers)

3 key-value pairs were sent over a network.


([('max', 185)], 'python max =', 185)

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
from typing import Iterator
import numpy as np

np.random.seed(7)
numbers = (np.random.randn(100) * 10 + 5).tolist()  # вещественные

maps = 4
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for x in split:
            yield (None, float(x))

    split_size = int(np.ceil(len(numbers) / maps))
    for i in range(0, len(numbers), split_size):
        yield RECORDREADER(numbers[i:i + split_size])

def MAP(_, x: float):
    # Храним (частичная_сумма, частичный_счётчик)
    yield ("avg", (x, 1))

def COMBINER_AVG(key: str, pairs: Iterator[tuple]):
    s = 0.0
    c = 0
    for (x, n) in pairs:
        s += x
        c += n
    yield (key, (s, c))

def REDUCE_AVG(key: str, pairs: Iterator[tuple]):
    s = 0.0
    c = 0
    for (partial_sum, partial_count) in pairs:
        s += partial_sum
        c += partial_count
    yield (key, s / c if c else float("nan"))

partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE_AVG, COMBINER=COMBINER_AVG
)
partitioned_output = [(pid, list(part)) for (pid, part) in partitioned_output]

result = [kv for (pid, part) in partitioned_output for kv in part][0][1]
result, "numpy mean =", float(np.mean(numbers))

4 key-value pairs were sent over a network.


(5.120654245177354, 'numpy mean =', 5.120654245177356)

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
import itertools

def groupbykey_sort(iterable):
    """groupByKey на основе сортировки.
    Возвращает итератор пар (k, [v1, v2, ...]) в порядке возрастания k.
    """
    data = sorted(iterable, key=lambda kv: kv[0])
    for k, group in itertools.groupby(data, key=lambda kv: kv[0]):
        yield (k, [v for (_, v) in group])

# Проверка на простом примере
pairs = [('b', 1), ('a', 2), ('b', 3), ('a', 4), ('c', 5), ('b', 6)]
print("Input:", pairs)

print("groupbykey (dict-based, порядок ключей не гарантирован):")
print(sorted(list(groupbykey(pairs)), key=lambda x: x[0]))

print("groupbykey_sort (sorting-based):")
print(list(groupbykey_sort(pairs)))

# Пример на WordCount map-выводе
example_docs = ["a b a", "b c", "a c c"]
def RR_wc():
    for (docid, doc) in enumerate(example_docs):
        yield (str(docid), doc)

def MAP_wc(_, doc):
    for w in doc.split():
        yield (w, 1)

map_out = list(flatten(map(lambda x: MAP_wc(*x), RR_wc())))
print("\nWordCount map output:", map_out)
print("Grouped:", list(groupbykey_sort(map_out)))

Input: [('b', 1), ('a', 2), ('b', 3), ('a', 4), ('c', 5), ('b', 6)]
groupbykey (dict-based, порядок ключей не гарантирован):
[('a', [2, 4]), ('b', [1, 3, 6]), ('c', [5])]
groupbykey_sort (sorting-based):
[('a', [2, 4]), ('b', [1, 3, 6]), ('c', [5])]

WordCount map output: [('a', 1), ('b', 1), ('a', 1), ('b', 1), ('c', 1), ('a', 1), ('c', 1), ('c', 1)]
Grouped: [('a', [1, 1, 1]), ('b', [1, 1]), ('c', [1, 1, 1])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
from typing import Iterator
import numpy as np

values = [5, 1, 5, 2, 2, 9, 1, 7, 7, 7, 3, 5]
maps = 3
reducers = 2

def INPUTFORMAT():
    global maps
    def RECORDREADER(split):
        for x in split:
            yield (None, x)
    split_size = int(np.ceil(len(values) / maps))
    for i in range(0, len(values), split_size):
        yield RECORDREADER(values[i:i + split_size])

def MAP(_, x: int):
    # Ключ — само значение; так reducer увидит все дубликаты под одним ключом
    yield (x, None)

def REDUCE_DISTINCT(x: int, _values: Iterator[None]):
    # Возвращаем ключ один раз => уникальные элементы
    yield (x, None)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE_DISTINCT)
partitioned_output = [(pid, list(part)) for (pid, part) in partitioned_output]

distinct_values = sorted([x for (pid, part) in partitioned_output for (x, _) in part])
distinct_values, "python set =", sorted(set(values))

12 key-value pairs were sent over a network.


([1, 2, 3, 5, 7, 9], 'python set =', [1, 2, 3, 5, 7, 9])

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
R = [
    ("Ivan", 25, "Samara"),
    ("Olga", 31, "Moscow"),
    ("Petr", 40, "Samara"),
    ("Anna", 19, "Kazan"),
    ("Dmitry", 31, "Samara"),
]

def C(t):
    # пример предиката: возраст >= 30
    return t[1] >= 30

def RECORDREADER():
    for t in R:
        yield (t, t)

def MAP(_, t):
    if C(t):
        yield (t, t)

def REDUCE(t, _values):
    yield (t, t)

selected = [t for (t, _) in MapReduce(RECORDREADER, MAP, REDUCE)]
selected

[('Olga', 31, 'Moscow'), ('Petr', 40, 'Samara'), ('Dmitry', 31, 'Samara')]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
R = [
    ("Ivan", 25, "Samara"),
    ("Olga", 31, "Moscow"),
    ("Petr", 40, "Samara"),
    ("Anna", 19, "Kazan"),
    ("Dmitry", 31, "Samara"),
]
S_idx = [2]  # проекция на город

def project(t, idxs):
    return tuple(t[i] for i in idxs)

def RECORDREADER():
    for t in R:
        yield (t, t)

def MAP(_, t):
    t2 = project(t, S_idx)
    yield (t2, t2)

def REDUCE(t2, _values):
    yield (t2, t2)

projection = [t2 for (t2, _) in MapReduce(RECORDREADER, MAP, REDUCE)]
projection

[('Samara',), ('Moscow',), ('Kazan',)]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
R = [("a", 1), ("b", 2), ("c", 3)]
S = [("b", 2), ("d", 4)]

def RECORDREADER():
    for t in R:
        yield ("R", t)
    for t in S:
        yield ("S", t)

def MAP(_, t):
    yield (t, t)

def REDUCE(t, _values):
    yield (t, t)

union = [t for (t, _) in MapReduce(RECORDREADER, MAP, REDUCE)]
sorted(union)

[('a', 1), ('b', 2), ('c', 3), ('d', 4)]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
R = [("a", 1), ("b", 2), ("c", 3), ("c", 3)]
S = [("b", 2), ("c", 3), ("c", 3), ("d", 4)]

def RECORDREADER():
    for t in R:
        yield ("R", t)
    for t in S:
        yield ("S", t)

def MAP(rel, t):
    yield (t, rel)

def REDUCE(t, rels):
    rels = set(rels)
    if ("R" in rels) and ("S" in rels):
        yield (t, t)

intersect = [t for (t, _) in MapReduce(RECORDREADER, MAP, REDUCE)]
sorted(intersect)

[('b', 2), ('c', 3)]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
R = [("a", 1), ("b", 2), ("c", 3)]
S = [("b", 2), ("d", 4), ("c", 3)]

def RECORDREADER():
    for t in R:
        yield ("R", t)
    for t in S:
        yield ("S", t)

def MAP(rel, t):
    # значение — имя отношения
    yield (t, rel)

def REDUCE(t, rels):
    rels = set(rels)
    if rels == {"R"}:
        yield (t, t)

diff = [t for (t, _) in MapReduce(RECORDREADER, MAP, REDUCE)]
sorted(diff)

[('a', 1)]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
R = [("x1", "b1"), ("x2", "b1"), ("x3", "b2")]
S = [("b1", "c7"), ("b1", "c8"), ("b3", "c9")]

def RECORDREADER():
    for (a, b) in R:
        yield ("R", (a, b))
    for (b, c) in S:
        yield ("S", (b, c))

def MAP(rel, t):
    if rel == "R":
        a, b = t
        yield (b, ("R", a))
    else:
        b, c = t
        yield (b, ("S", c))

def REDUCE(b, tagged_values):
    left = [a for (tag, a) in tagged_values if tag == "R"]
    right = [c for (tag, c) in tagged_values if tag == "S"]
    for a in left:
        for c in right:
            yield (None, (a, b, c))

join_result = [row for (_, row) in MapReduce(RECORDREADER, MAP, REDUCE)]
sorted(join_result)

[('x1', 'b1', 'c7'),
 ('x1', 'b1', 'c8'),
 ('x2', 'b1', 'c7'),
 ('x2', 'b1', 'c8')]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
R = [
    ("g1", 10, "x"),
    ("g1", 5, "y"),
    ("g2", 7, "x"),
    ("g2", 3, "z"),
    ("g2", 100, "q"),
]

def RECORDREADER():
    for t in R:
        yield (None, t)

def MAP(_, t):
    a, b, c = t
    yield (a, b)

def REDUCE(a, bs):
    # пример θ = SUM
    s = 0
    for b in bs:
        s += b
    yield (a, s)

grouped = list(MapReduce(RECORDREADER, MAP, REDUCE))
sorted(grouped)

[('g1', 15), ('g2', 110)]

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
import numpy as np
from typing import Iterator

# Матрица M (I x J) и вектор v (J)
np.random.seed(1)
I, J = 4, 5
M = np.random.randint(0, 5, size=(I, J)).astype(float)
v = np.random.randn(J).astype(float)

# join по j и вычисление частичных произведений (i, m_ij * v_j)
def RECORDREADER1():
    for i in range(I):
        for j in range(J):
            yield (("M", i, j), float(M[i, j]))
    for j in range(J):
        yield (("V", j), float(v[j]))

def MAP1(k1, val):
    tag = k1[0]
    if tag == "M":
        _, i, j = k1
        yield (j, ("M", i, val))
    else:
        _, j = k1
        yield (j, ("V", val))

def REDUCE1(j, tagged_values: Iterator[tuple]):
    vj = None
    m_records = []
    for rec in tagged_values:
        if rec[0] == "V":
            vj = rec[1]
        else:
            # ("M", i, m_ij)
            m_records.append((rec[1], rec[2]))
    if vj is None:
        return
    for (i, mij) in m_records:
        yield (i, mij * vj)

partials = list(MapReduce(RECORDREADER1, MAP1, REDUCE1))

# суммирование по i
def RECORDREADER2():
    for (i, val) in partials:
        yield (i, val)

def MAP2(i, val):
    yield (i, val)

def REDUCE2(i, vals: Iterator[float]):
    s = 0.0
    for x in vals:
        s += x
    yield (i, s)

result = list(MapReduce(RECORDREADER2, MAP2, REDUCE2))
result_sorted = [val for (i, val) in sorted(result, key=lambda x: x[0])]

reference = M @ v
print("Matches numpy:", np.allclose(result_sorted, reference))
result_sorted, reference.tolist()

Matches numpy: True


([5.2135888013917615, 3.150816560351816, 7.012649990043872, 5.522147470748994],
 [5.213588801391762, 3.150816560351816, 7.0126499900438715, 5.522147470748994])

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
  (i, k) = key
  s = 0.0
  for x in values:
    s += x
  yield ((i, k), s)

Проверьте своё решение

In [ ]:
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution))

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
import numpy as np
from typing import Iterator

np.random.seed(2)
I, J, K = 3, 4, 5
A = np.random.randn(I, J)
B = np.random.randn(J, K)

# join по j и выдача частичных произведений (i,k, a_ij*b_jk)
def RECORDREADER1():
    # сначала A, потом B
    for i in range(I):
        for j in range(J):
            yield (("A", i, j), float(A[i, j]))
    for j in range(J):
        for k in range(K):
            yield (("B", j, k), float(B[j, k]))

def MAP1(k1, val):
    tag = k1[0]
    if tag == "A":
        _, i, j = k1
        yield (j, ("A", i, val))
    else:
        _, j, k = k1
        yield (j, ("B", k, val))

def REDUCE1(j, tagged_values: Iterator[tuple]):
    left = [(i, aij) for (tag, i, aij) in tagged_values if tag == "A"]
    right = [(k, bjk) for (tag, k, bjk) in tagged_values if tag == "B"]
    for (i, aij) in left:
        for (k, bjk) in right:
            yield ((i, k), aij * bjk)

partials = list(MapReduce(RECORDREADER1, MAP1, REDUCE1))

# суммирование по (i,k)
def RECORDREADER2():
    for ((i, k), val) in partials:
        yield ((i, k), val)

def MAP2(key, val):
    yield (key, val)

def REDUCE2(key, vals: Iterator[float]):
    s = 0.0
    for x in vals:
        s += x
    yield (key, s)

P = list(MapReduce(RECORDREADER2, MAP2, REDUCE2))

# Проверка
def asmatrix(reduce_output, I, K):
    mat = np.zeros((I, K), dtype=float)
    for ((i, k), v) in reduce_output:
        mat[i, k] = v
    return mat

reference = A @ B
solution = asmatrix(P, I, K)
print("Matches numpy:", np.allclose(reference, solution))
solution

Matches numpy: True


array([[-2.57951602,  0.28902934,  0.24685186,  0.36107171,  5.03897679],
       [ 0.54439154,  3.90703195, -0.95360489,  2.02459989, -2.93337197],
       [-3.69279933, -1.94045963, -1.11718963,  0.68193608,  4.9255584 ]])

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
import numpy as np
from typing import Iterator
import math

np.random.seed(3)
I, J, K = 3, 4, 6
A = np.random.randn(I, J)
B = np.random.randn(J, K)

maps = 4
reducers = 3

def INPUTFORMAT1():
    # Два RECORDREADER-а: один выдаёт элементы A, второй элементы B
    def RR_A():
        for i in range(I):
            for j in range(J):
                yield (("A", i, j), float(A[i, j]))
    def RR_B():
        for j in range(J):
            for k in range(K):
                yield (("B", j, k), float(B[j, k]))
    yield RR_A()
    yield RR_B()

def MAP1(k1, val):
    tag = k1[0]
    if tag == "A":
        _, i, j = k1
        yield (j, ("A", i, val))
    else:
        _, j, k = k1
        yield (j, ("B", k, val))

def REDUCE1(j, tagged_values: Iterator[tuple]):
    left = [(i, aij) for (tag, i, aij) in tagged_values if tag == "A"]
    right = [(k, bjk) for (tag, k, bjk) in tagged_values if tag == "B"]
    for (i, aij) in left:
        for (k, bjk) in right:
            yield ((i, k), aij * bjk)

stage1 = MapReduceDistributed(INPUTFORMAT1, MAP1, REDUCE1, COMBINER=None)
stage1 = [(pid, list(part)) for (pid, part) in stage1]
partials = [kv for (pid, part) in stage1 for kv in part]

# суммирование по (i,k)
maps = 3
reducers = 3

def INPUTFORMAT2():
    global maps
    split_size = int(math.ceil(len(partials) / maps)) if partials else 1
    def RR(split):
        for kv in split:
            yield kv
    for i in range(0, len(partials), split_size):
        yield RR(partials[i:i + split_size])

def MAP2(key, val):
    yield (key, val)

def REDUCE2(key, vals: Iterator[float]):
    s = 0.0
    for x in vals:
        s += x
    yield (key, s)

stage2 = MapReduceDistributed(INPUTFORMAT2, MAP2, REDUCE2, COMBINER=None)
stage2 = [(pid, list(part)) for (pid, part) in stage2]
P = [kv for (pid, part) in stage2 for kv in part]

def asmatrix(reduce_output, I, K):
    mat = np.zeros((I, K), dtype=float)
    for ((i, k), v) in reduce_output:
        mat[i, k] = v
    return mat

reference = A @ B
solution = asmatrix(P, I, K)
print("Matches numpy:", np.allclose(reference, solution))

36 key-value pairs were sent over a network.
72 key-value pairs were sent over a network.
Matches numpy: True


Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
import numpy as np
import math
from typing import Iterator
import random

np.random.seed(4)
random.seed(4)

I, J, K = 4, 5, 3
A = np.random.randn(I, J)
B = np.random.randn(J, K)

maps = 6
reducers = 3

# Разбиваем элементы каждой матрицы на несколько RECORDREADER-ов (сплитов)
A_entries = [(("A", i, j), float(A[i, j])) for i in range(I) for j in range(J)]
B_entries = [(("B", j, k), float(B[j, k])) for j in range(J) for k in range(K)]

def chunk(lst, n_chunks):
    n = len(lst)
    split_size = int(math.ceil(n / n_chunks))
    return [lst[i:i + split_size] for i in range(0, n, split_size)]

A_splits = chunk(A_entries, n_chunks=3)
B_splits = chunk(B_entries, n_chunks=3)

def INPUTFORMAT1():
    def RR(split):
        for kv in split:
            yield kv
    for split in A_splits:
        yield RR(split)
    for split in B_splits:
        yield RR(split)

def MAP1(k1, val):
    tag = k1[0]
    if tag == "A":
        _, i, j = k1
        yield (j, ("A", i, val))
    else:
        _, j, k = k1
        yield (j, ("B", k, val))

def REDUCE1(j, tagged_values: Iterator[tuple]):
    left = [(i, aij) for (tag, i, aij) in tagged_values if tag == "A"]
    right = [(k, bjk) for (tag, k, bjk) in tagged_values if tag == "B"]
    for (i, aij) in left:
        for (k, bjk) in right:
            yield ((i, k), aij * bjk)

stage1 = MapReduceDistributed(INPUTFORMAT1, MAP1, REDUCE1, COMBINER=None)
stage1 = [(pid, list(part)) for (pid, part) in stage1]
partials = [kv for (pid, part) in stage1 for kv in part]

# суммирование
maps = 4
reducers = 3

def INPUTFORMAT2():
    global maps
    split_size = int(math.ceil(len(partials) / maps)) if partials else 1
    def RR(split):
        for kv in split:
            yield kv
    for i in range(0, len(partials), split_size):
        yield RR(partials[i:i + split_size])

def MAP2(key, val):
    yield (key, val)

def REDUCE2(key, vals: Iterator[float]):
    s = 0.0
    for x in vals:
        s += x
    yield (key, s)

stage2 = MapReduceDistributed(INPUTFORMAT2, MAP2, REDUCE2, COMBINER=None)
stage2 = [(pid, list(part)) for (pid, part) in stage2]
P = [kv for (pid, part) in stage2 for kv in part]

def asmatrix(reduce_output, I, K):
    mat = np.zeros((I, K), dtype=float)
    for ((i, k), v) in reduce_output:
        mat[i, k] = v
    return mat

reference = A @ B
solution = asmatrix(P, I, K)
print("Full coverage (splits cover the whole matrices) matches numpy:", np.allclose(reference, solution))

B_entries_subset = [kv for kv in B_entries if random.random() > 0.3]
B_splits_subset = chunk(B_entries_subset, n_chunks=3)

def INPUTFORMAT1_subset():
    def RR(split):
        for kv in split:
            yield kv
    for split in A_splits:
        yield RR(split)
    for split in B_splits_subset:
        yield RR(split)

stage1s = MapReduceDistributed(INPUTFORMAT1_subset, MAP1, REDUCE1, COMBINER=None)
stage1s = [(pid, list(part)) for (pid, part) in stage1s]
partials_s = [kv for (pid, part) in stage1s for kv in part]

def INPUTFORMAT2s():
    global maps
    split_size = int(math.ceil(len(partials_s) / maps)) if partials_s else 1
    def RR(split):
        for kv in split:
            yield kv
    for i in range(0, len(partials_s), split_size):
        yield RR(partials_s[i:i + split_size])

stage2s = MapReduceDistributed(INPUTFORMAT2s, MAP2, REDUCE2, COMBINER=None)
stage2s = [(pid, list(part)) for (pid, part) in stage2s]
P_s = [kv for (pid, part) in stage2s for kv in part]
solution_subset = asmatrix(P_s, I, K)

print("Subset coverage equals full numpy product:", np.allclose(reference, solution_subset))

35 key-value pairs were sent over a network.
60 key-value pairs were sent over a network.
Full coverage (splits cover the whole matrices) matches numpy: True
26 key-value pairs were sent over a network.
24 key-value pairs were sent over a network.
Subset coverage equals full numpy product: False
